In [22]:
from pathlib import Path
import polars as pl
import rasterio

from tqdm import tqdm

In [23]:
new = pl.read_csv("../data/id_mapping.csv").filter(pl.col.dataset.ne("Evoland"))

In [24]:
old = pl.read_csv(
    "https://huggingface.co/datasets/JR-DIGITAL/DISFOR/resolve/main/id_mapping.csv"
).filter(pl.col.dataset.ne("Evoland"))

In [25]:
joined = new.join(old, on=["dataset", "original_sample_id"], how="full")
joined

sample_id,original_sample_id,dataset,sample_id_right,original_sample_id_right,dataset_right
i64,i64,str,i64,i64,str
913,1,"""HRVPP""",1006,1,"""HRVPP"""
914,2,"""HRVPP""",1007,2,"""HRVPP"""
915,3,"""HRVPP""",1008,3,"""HRVPP"""
916,4,"""HRVPP""",1009,4,"""HRVPP"""
917,5,"""HRVPP""",1010,5,"""HRVPP"""
…,…,…,…,…,…
null,null,null,3115,2110,"""HRVPP"""
null,null,null,1522,517,"""HRVPP"""
null,null,null,2014,1009,"""HRVPP"""


In [29]:
joined.filter(pl.col.original_sample_id_right.is_null())

sample_id,original_sample_id,dataset,sample_id_right,original_sample_id_right,dataset_right
i64,i64,str,i64,i64,str
922,10,"""HRVPP""",null,null,null
930,18,"""HRVPP""",null,null,null
952,40,"""HRVPP""",null,null,null
996,84,"""HRVPP""",null,null,null
1025,114,"""HRVPP""",null,null,null
…,…,…,…,…,…
3052,2218,"""HRVPP""",null,null,null
3081,2249,"""HRVPP""",null,null,null
3164,2334,"""HRVPP""",null,null,null


In [35]:
joined = new.join(old, on=["dataset", "original_sample_id"], how="inner")

In [ ]:
source = Path(r"C:\Users\Jonas.Viehweger\AppData\Local\disfor\disfor\Cache\0.1.0\tiffs")
destination = Path("../data/tiffs/")
for row in tqdm(joined.iter_rows(named=True), total=len(joined)):
    sample_out = destination / str(row["sample_id"])
    sample_out.mkdir(parents=True, exist_ok=True)
    for tiff in (source / str(row["sample_id_right"])).glob("*.tif"):
        with rasterio.open(tiff) as src:
            profile = src.profile
            profile.update(compress="zstd")
            arr = src.read()
            out_path = sample_out / tiff.name
            with rasterio.open(out_path, "w", **profile) as dst:
                dst.write(arr)

16it [04:38, 17.38s/it]


KeyboardInterrupt: 